In [2]:
%matplotlib inline
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
import os

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
import graduation_pytorch as gra

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
from torchvision import transforms, datasets

## 预处理

In [23]:
# 指定RGB三个通道的均值和方差来将图像通道归一化
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_augs = transforms.Compose([
        transforms.RandomResizedCrop(size=224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

test_augs = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize
    ])

In [19]:
train_dataset = datasets.ImageFolder(root='../数据集/Google dataset of SIRI-WHU_earth_im_tiff/data/train', transform=train_augs)
test_dataset = datasets.ImageFolder(root='../数据集/Google dataset of SIRI-WHU_earth_im_tiff/data/test', transform=test_augs)

In [20]:
train_dataset,test_dataset  # 2400*0.8

(Dataset ImageFolder
     Number of datapoints: 1920
     Root Location: ../数据集/Google dataset of SIRI-WHU_earth_im_tiff/data/train
     Transforms (if any): Compose(
                              RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                              RandomHorizontalFlip(p=0.5)
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          )
     Target Transforms (if any): None,
 Dataset ImageFolder
     Number of datapoints: 480
     Root Location: ../数据集/Google dataset of SIRI-WHU_earth_im_tiff/data/test
     Transforms (if any): Compose(
                              Resize(size=256, interpolation=PIL.Image.BILINEAR)
                              CenterCrop(size=(224, 224))
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 

In [21]:
feature, label = train_dataset[450]
feature, label

(tensor([[[-0.9020, -0.9020, -0.9020,  ..., -0.9363, -0.9363, -0.9363],
          [-0.9020, -0.9020, -0.9020,  ..., -0.9363, -0.9363, -0.9363],
          [-0.8849, -0.9020, -0.9192,  ..., -0.9192, -0.9192, -0.9192],
          ...,
          [-0.9534, -0.9534, -0.9534,  ..., -0.7308, -0.8849, -1.0048],
          [-0.9534, -0.9534, -0.9534,  ..., -0.9877, -1.0048, -0.9877],
          [-0.9534, -0.9534, -0.9534,  ..., -1.1760, -1.0733, -0.9705]],
 
         [[-0.8803, -0.8803, -0.8803,  ..., -0.8978, -0.8978, -0.8978],
          [-0.8803, -0.8803, -0.8803,  ..., -0.8978, -0.8978, -0.8978],
          [-0.8627, -0.8803, -0.8978,  ..., -0.8803, -0.8803, -0.8803],
          ...,
          [-0.8803, -0.8803, -0.8803,  ..., -0.7752, -0.9678, -1.0903],
          [-0.8803, -0.8803, -0.8803,  ..., -1.0378, -1.0903, -1.0728],
          [-0.8803, -0.8803, -0.8803,  ..., -1.2304, -1.1604, -1.0728]],
 
         [[-0.5844, -0.5844, -0.5844,  ..., -0.6541, -0.6541, -0.6541],
          [-0.5844, -0.5844,

In [22]:
# """将训练集划分为训练集和验证集"""
# train_db,val_db,test_db = torch.utils.data.random_split(dataset, [1920, 240, 240])

In [24]:
pretrained_net = models.resnet34(pretrained=True)

In [25]:
pretrained_net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [26]:
#打印源模型的成员变量fc。作为一个全连接层，它将ResNet最终的全局平均池化层输出变换成ImageNet数据集上1000类的输出。
print(pretrained_net.fc)

Linear(in_features=512, out_features=1000, bias=True)


In [27]:
# 所以我们应该将最后的fc成修改我们需要的输出类别数:
pretrained_net.fc = nn.Linear(512, 12)
print(pretrained_net.fc)
#此时，pretrained_net的fc层就被随机初始化了，但是其他层依然保存着预训练得到的参数。

Linear(in_features=512, out_features=12, bias=True)


In [28]:
# 由于是在很大的ImageNet数据集上预训练的，所以参数已经足够好，因此一般只需使用较小的学习率来微调这些参数，
#而fc中的随机初始化参数一般需要更大的学习率从头训练。PyTorch可以方便的对模型的不同部分设置不同的学习参数，
#我们在下面代码中将fc的学习率设为已经预训练过的部分的10倍。

In [29]:
output_params = list(map(id, pretrained_net.fc.parameters()))
feature_params = filter(lambda p: id(p) not in output_params, pretrained_net.parameters()) # 除fc层的参数

lr = 0.01
optimizer = optim.SGD([{'params': feature_params},
                       {'params': pretrained_net.fc.parameters(), 'lr': lr * 10}],
                       lr=lr, weight_decay=0.001)

In [30]:
train_iter = DataLoader(train_dataset,
                            batch_size = 64, shuffle=True)
test_iter = DataLoader(test_dataset,
                            batch_size = 64, shuffle=False)

In [50]:
# data_dir = '../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif'
# loader = gra.fetch_dataloaders(data_dir, [0.6, 0.2, 0.2], batchsize=64)

In [31]:
len(train_iter)

30

In [32]:
1920/64

30.0

In [35]:
def train_fine_tuning(net, optimizer, batch_size=32, num_epochs=5):
    loss = torch.nn.CrossEntropyLoss()
    d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

In [36]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 5)

training on  cuda
epoch 1, loss 3.6254, train acc 0.407, test acc 0.887, time 75.9 sec
epoch 2, loss 0.3104, train acc 0.795, test acc 0.894, time 58.8 sec
epoch 3, loss 0.1397, train acc 0.859, test acc 0.933, time 43.8 sec
epoch 4, loss 0.0925, train acc 0.868, test acc 0.960, time 31.4 sec
epoch 5, loss 0.0650, train acc 0.890, test acc 0.967, time 31.0 sec


In [37]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 3)

training on  cuda
epoch 1, loss 0.2687, train acc 0.916, test acc 0.938, time 32.1 sec
epoch 2, loss 0.1277, train acc 0.907, test acc 0.963, time 31.1 sec
epoch 3, loss 0.0816, train acc 0.920, test acc 0.965, time 30.9 sec


In [38]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 3)

training on  cuda
epoch 1, loss 0.2123, train acc 0.931, test acc 0.963, time 31.3 sec
epoch 2, loss 0.1258, train acc 0.923, test acc 0.954, time 31.2 sec
epoch 3, loss 0.0712, train acc 0.931, test acc 0.958, time 30.9 sec


In [39]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 8)

training on  cuda
epoch 1, loss 0.1796, train acc 0.944, test acc 0.952, time 30.8 sec
epoch 2, loss 0.0816, train acc 0.944, test acc 0.973, time 30.8 sec
epoch 3, loss 0.0510, train acc 0.953, test acc 0.967, time 32.2 sec
epoch 4, loss 0.0388, train acc 0.947, test acc 0.963, time 32.6 sec
epoch 5, loss 0.0309, train acc 0.952, test acc 0.973, time 31.1 sec
epoch 6, loss 0.0248, train acc 0.954, test acc 0.956, time 31.6 sec
epoch 7, loss 0.0209, train acc 0.956, test acc 0.958, time 31.8 sec
epoch 8, loss 0.0185, train acc 0.953, test acc 0.965, time 31.9 sec


In [40]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 5)

training on  cuda
epoch 1, loss 0.1341, train acc 0.951, test acc 0.971, time 32.1 sec
epoch 2, loss 0.0756, train acc 0.953, test acc 0.971, time 32.2 sec
epoch 3, loss 0.0378, train acc 0.964, test acc 0.963, time 31.3 sec
epoch 4, loss 0.0310, train acc 0.958, test acc 0.973, time 30.9 sec
epoch 5, loss 0.0247, train acc 0.956, test acc 0.973, time 31.1 sec


In [41]:
pretrained_net.fc = nn.Sequential()
PATH = "./my_model/remote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)
# net2 = MLP()
# net2.load_state_dict(torch.load(PATH))
# Y2 = net2(X)
# Y2 == Y

In [42]:
def load_query_image(img_path):
    test_augs = transforms.Compose([
            transforms.Resize(size=256),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(),
            normalize
        ])
    query_image = datasets.folder.default_loader(img_path)
    query_image = test_augs(query_image)
    query_image = query_image.view(-1,3,224,224) # 3,224,224 是预设的 要输入给网络的
    return query_image

In [43]:
# 封装距离比较函数
def similarity_of_two(img1, img2):
    # 加载到gpu
    img1 = img1.to(device)
    img2 = img2.to(device)
    # 提取特征（1,3,224,224）=> (1,512)
    output_img1 = pretrained_net(img1)
    output_img2 = pretrained_net(img2)
    # (1,512) => (512)
    output_img1 = output_img1.view(512)
    output_img2 = output_img2.view(512)
    output_img1 = output_img1.data.cpu()
    output_img2 = output_img2.data.cpu()
    out = (output_img1*output_img2)/(torch.norm(output_img1,p=2,dim=0, keepdim=True)*torch.norm(output_img2,p=2,dim=0, keepdim=True))
    return out.sum()

In [44]:
pwd

'D:\\360downloads\\毕业论文\\pytorch'

In [45]:
pretrained_net.fc = nn.Sequential()

In [46]:
pretrained_net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [47]:
agriculture1 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/agriculture/0001.tif')
agriculture2 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/agriculture/0002.tif')
agriculture3 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/agriculture/0034.tif')
agriculture4 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/agriculture/0166.tif')
park1 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/park/0001.tif')
park2 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/park/0135.tif')
park3 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/park/0179.tif')
water1 = load_query_image('../数据集/Google dataset of SIRI-WHU_earth_im_tiff/12class_tif/water/0001.tif')

In [48]:
dif1 = similarity_of_two(agriculture1, agriculture2)
dif2 = similarity_of_two(agriculture1, agriculture3)
dif3 = similarity_of_two(agriculture1, agriculture4)

In [49]:
dif4 = similarity_of_two(agriculture1, park1)
dif5 = similarity_of_two(agriculture1, park2)
dif6 = similarity_of_two(agriculture1, park3)

In [50]:
dif7 = similarity_of_two(agriculture1, water1)

In [51]:
dif1, dif2, dif3

(tensor(0.8294), tensor(0.5850), tensor(0.6584))

In [52]:
dif4, dif5, dif6

(tensor(0.5466), tensor(0.5000), tensor(0.6092))

In [53]:
dif7

tensor(0.4561)